In [0]:
!pip install tensorflow --upgrade

     |████████████████████████████████| 421.8MB 45kB/s 
     |████████████████████████████████| 3.9MB 38.3MB/s 
     |████████████████████████████████| 450kB 46.2MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

#tf.enable_eager_execution()

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Data Cleaning

In [0]:
# read the data files

df=pd.read_csv("drive/My Drive/Colab Notebooks/articles1.csv")

df2=pd.read_csv("drive/My Drive/Colab Notebooks/articles2.csv")

df3=pd.read_csv("drive/My Drive/Colab Notebooks/articles3.csv")

In [0]:
## combine the dfs

df=pd.concat([df,df2])

df=pd.concat([df,df3])

In [0]:
df.head(3)

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."


In [0]:
## create a list containing the text of the articles

y=[]
for i in df.loc[:,"content"].values:
    y.append(i)

In [0]:
y[0:4]

['WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been

In [0]:
## create a new input list containing articles over 5000 in length to avoid getting some non-english and special characters

newinput=[]
for i in y:
    if len(i)<1000:
        newinput.append(i)

In [0]:
len(newinput)

10826

In [0]:
concatstr = ' '.join(newinput) ## take the contents of the list and put it into one giant string

In [0]:
len(concatstr)

6931479

Can Change This Number of Characters:

In [0]:
newinput = concatstr[0:2000000] # work with the first 30 million characters
text = newinput

In [0]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

152 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print('{')
for char,_ in zip(char2idx, range(120)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '#' :   2,
  '$' :   3,
  '%' :   4,
  '&' :   5,
  '(' :   6,
  ')' :   7,
  '*' :   8,
  '+' :   9,
  ',' :  10,
  '.' :  11,
  '0' :  12,
  '1' :  13,
  '2' :  14,
  '3' :  15,
  '4' :  16,
  '5' :  17,
  '6' :  18,
  '7' :  19,
  '8' :  20,
  '9' :  21,
  ':' :  22,
  '?' :  23,
  '@' :  24,
  'A' :  25,
  'B' :  26,
  'C' :  27,
  'D' :  28,
  'E' :  29,
  'F' :  30,
  'G' :  31,
  'H' :  32,
  'I' :  33,
  'J' :  34,
  'K' :  35,
  'L' :  36,
  'M' :  37,
  'N' :  38,
  'O' :  39,
  'P' :  40,
  'Q' :  41,
  'R' :  42,
  'S' :  43,
  'T' :  44,
  'U' :  45,
  'V' :  46,
  'W' :  47,
  'X' :  48,
  'Y' :  49,
  'Z' :  50,
  '[' :  51,
  ']' :  52,
  '_' :  53,
  '`' :  54,
  'a' :  55,
  'b' :  56,
  'c' :  57,
  'd' :  58,
  'e' :  59,
  'f' :  60,
  'g' :  61,
  'h' :  62,
  'i' :  63,
  'j' :  64,
  'k' :  65,
  'l' :  66,
  'm' :  67,
  'n' :  68,
  'o' :  69,
  'p' :  70,
  'q' :  71,
  'r' :  72,
  's' :  73,
  't' :  74,
  'u' :  75,
  'v' :  7

In [0]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:15]), text_as_int[:15]))

'LONDON  —   Que' ---- characters mapped to int ---- > [ 36  39  38  28  39  38   0   0 134   0   0   0  41  75  59]


## Model Building

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'LONDON  —   Queen Elizabeth II, who has been battling a cold for more than a week, missed a New Year’'
's Day church service at her country estate in Sandringham, Buckingham Palace said on Sunday. A week e'
'arlier, the queen, who is 90, missed a Christmas Day church service, for the first time since 1988, b'
'ecause of the illness. “The Queen does not yet feel ready to attend church as she is still recuperati'
'ng from a heavy cold,” the palace said in a statement. The queen’s husband, Prince Philip, who had al'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)
#model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) # './training_checkpoints/ckpt_30'
#model.load_weights('./training_checkpoints/ckpt_100')

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 152) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (128, None, 256)          38912     
_________________________________________________________________
gru_14 (GRU)                 (128, None, 1024)         3938304   
_________________________________________________________________
gru_15 (GRU)                 (128, None, 1024)         6297600   
_________________________________________________________________
gru_16 (GRU)                 (128, None, 1024)         6297600   
_________________________________________________________________
dense_12 (Dense)             (128, None, 152)          155800    
Total params: 16,728,216
Trainable params: 16,728,216
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([138, 150,  15,  35,  52,  70, 132,  72, 127,  48, 122,  76,  38,
        90, 148,  85,  33, 144,  90,  42,  39,  83,  52,  89,  48, 133,
        17, 139, 121,  45,  88, 115,   5,   5,  40,  52,  81,  24,  44,
       120,  68,  45,   4, 129,   5, 127, 151,  45,  85,  56,  98, 117,
       113,  37,  86,  90,  16,  53,  80,  33,  37,  77, 102,  43, 139,
        29,  12,  35,  37,  61,  22,  39,  74, 112,  15, 150,  49,   0,
        29,  66,  75,  56,  98,  51,  21,  21,  86,  68,  12,  16,  72,
       124,  85,  45,  41, 147, 142,  84,  81,  59])

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'er baseball player than he was a football player. I think Urban Meyer kept Tim Tebow from playing ba'

Next Char Predictions: 
 '”😄3K]p\u2009rᴀXلvNé👉°I🇦éRO£]èX\u200e5•قUáش&&P]|@TفnU%ᴇ&ᴀ🙋U°băطرM¼é4_zIMwآS•E0KMg:Otد3😄Y Elubă[99¼n04rن°UQ🇺″®|e'


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Train Model

In [0]:
EPOCHS=100

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 154 steps
Epoch 1/100
154/154 [==============================] - 128s 834ms/step - loss: 2.8233
Epoch 2/100
154/154 [==============================] - 126s 817ms/step - loss: 1.7050
Epoch 3/100
154/154 [==============================] - 126s 817ms/step - loss: 1.3759
Epoch 4/100
154/154 [==============================] - 126s 817ms/step - loss: 1.2584
Epoch 5/100
154/154 [==============================] - 126s 817ms/step - loss: 1.1883
Epoch 6/100
154/154 [==============================] - 126s 817ms/step - loss: 1.1360
Epoch 7/100
154/154 [==============================] - 125s 814ms/step - loss: 1.0926
Epoch 8/100
154/154 [==============================] - 126s 817ms/step - loss: 1.0542
Epoch 9/100
154/154 [==============================] - 126s 816ms/step - loss: 1.0175
Epoch 10/100
154/154 [==============================] - 126s 818ms/step - loss: 0.9834
Epoch 11/100
154/154 [==============================] - 125s 815ms/step - loss: 0.9492
Epoch 12/100
154/154 [==========

KeyboardInterrupt: ignored

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

In [0]:
from google.colab import files
files.download('./training_checkpoints/ckpt_100.index') 
files.download('./training_checkpoints/ckpt_100.data-00000-of-00002') 
files.download('./training_checkpoints/ckpt_100.data-00001-of-00002')

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) # './training_checkpoints/ckpt_30'
model.load_weights('./training_checkpoints/ckpt_61')

model.build(tf.TensorShape([1, None]))

## Text Generating

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 500

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in tqdm(range(num_generate)):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"The market closed at an all time high today, boding good signs for the future of the economy."))

print(generate_text(model, start_string=u"Yesterday in Russia, the 2018 FIFA World Cup concluded with a nail biter of a final match. The score in the end was 3 goals to 2."))

print(generate_text(model, start_string=u"The news coming from Washington was a shock to everyone across the nation. There will be long lasting impacts of the legislation signed into law today, declared the Speaker of the House."))



  1%|          | 3/500 [00:00<00:16, 29.92it/s]

The market closed at an all time high today, boding good signs for the future of the economy. The only of the votes of margins: She neere to say hypocritical, that is the daily braim of very supportive. As you know, which makes me make sure we do  Freme misogyness. ” (  Grabien) Follow Ian Hanchett on Twitter @IanHanchett During testimony was “after being passed. ”  Tuesday, Breitbart senior political analyst Matt Boyle on Twitter @pamkeyNEN advertisement Conservative and destroy the   prodence sprinned in at Bloomberg. “Yes, so flower rough in Iowa was anounded” people were being chadg


  0%|          | 2/500 [00:00<00:26, 18.76it/s]

Yesterday in Russia, the 2018 FIFA World Cup concluded with a nail biter of a final match. The score in the end was 3 goals to 2. 0 did in place at major U. S. according to police, a woman sprints into the room shares will be speaking before President Donald Trump’s executive orders.  During an int Vice in Investigative Raddell in Belgian Airliness and another women over the arresor on the conclusion of the ancient racist executive creations. Florida government would be voted on social media: #myabulor before a conversation about a meeting at 44. 2 million regulations, and I think I goes thrown into the pandors, even long


100%|██████████| 500/500 [00:07<00:00, 63.71it/s]

The news coming from Washington was a shock to everyone across the nation. There will be long lasting impacts of the legislation signed into law today, declared the Speaker of the House. President, Voters Weight Tech convention since Monica Lewinsky.  The event was coverage out an update of meetings will be willing to jail on ABC’s “This Week,” Fively work on the relationships in Istanbul was a refugees at work Times columnist David Brooks argued that you never toured: No is so cowardick at all. From the Revolutionary Communist Parm in Pennsylvania, GOP presidential candidate Texas Senator Chuck Smith said President Donald Trump “will be insistent of the attack, but no other in


  1%|          | 4/500 [00:00<00:13, 35.58it/s]

The market closed at an all time high today, boding good signs for the future of the economy. ” Follow John Nolte on Twitter @NolteNC                     Donald Trump’s approval rants suspect Live from London, Rome, and Jerusalem, Breitbart    Alex Marlow will discuss the full speech due Crom, in ornkerman wound devaction of Clinton’s day in the victim, this is everything that will do that? MO. [Andrew Jackson in her effort. Trump (@realDonaldTrump) January 27, 2016,  Teld the residents “necle” is not a “high speed of foses. ” State Department to report that the Trump surprise for Democ


  0%|          | 2/500 [00:00<00:26, 18.91it/s]

Yesterday in Russia, the 2018 FIFA World Cup concluded with a nail biter of a final match. The score in the end was 3 goals to 2. 5 died in 1979 and said, “You know what, the temptate to give him hith Marco Rubio “bringing racist security measures in baseball.  Somali system was exposed her for the Vataness views with former Gov. Mitt Romney also “now any of that as for the extremists consider mainstream media to the Atlanta West. Vox, Orio Panel discuss the country’s unprecedented damage in particular in the week protested by the president. [From the Huffington Post:  Not sice is over Twitter and Facebook Chief over poli


100%|██████████| 500/500 [00:07<00:00, 67.98it/s]

The news coming from Washington was a shock to everyone across the nation. There will be long lasting impacts of the legislation signed into law today, declared the Speaker of the House. As far as wife as the hopes will back and that our poor injustice in these inees. Barring police of white voters had the beat Trump doesn’t apply to him. And I believe that view of it’s because the man will likely through the same second in Washington. D. C. on Friday approved a bill tim is wy stands for. ” She argued this week not to you are in this new Bill Maher added. The Clinton campaign accused Trump of reacting poorly to nine percent are racism. Amarkley will deterte for which message to


In [0]:
print(generate_text(model, start_string=u"A Chase bank near the University of Texas at Austin campus was robbed today"))


100%|██████████| 500/500 [00:07<00:00, 68.15it/s]

A Chase bank near the University of Texas at Austin campus was robbed today than Trump was getting more so by his latest conservative agenda.  When asked why America is facing their voices, he is not an issue at a gay people in the room undergo and do the subway system. ” The column endide an ouspi or slailing day for empty while owher this country has been arrested. [Valerie Starushok, McCarthy Spoke Battley were ullegedly when it counts us being to run for two hours. The event begins at 7pm, in an election is one issue of equal on a border wall as part of a crooked p


A Chase bank near the University of Texas at Austin campus was robbed today than Trump was getting more so by his latest conservative agenda.  When asked why America is facing their voices, he is not an issue at a gay people in the room undergo and do the subway system. ” The column endide an ouspi or slailing day for empty while owher this country has been arrested. [Valerie Starushok, McCarthy Spoke Battley were ullegedly when it counts us being to run for two hours. The event begins at 7pm, in an election is one issue of equal on a border wall as part of a crooked p


